# Evaluation metrics

## Geometry

### Chamfer distance
A standard metric to measure the shape dissimilarity between point clouds in point cloud completion.
The chamfer distance is defined as the sum of the distances from each point in A to its nearest neighbor in B, plus the sum of the distances from each point in B to its nearest neighbor in A

In [5]:
import open3d as o3d
import numpy as np

def get_chamfer_distance(pcd1,pcd2):
    # Calculate distances of pcd1 to pcd2. 
    dist_pc1_pc2 = pcd1.compute_point_cloud_distance(pcd2)
    dist_pc1_pc2 = np.asarray(dist_pc1_pc2)

    # Calculate distances of pcd2 to pcd1. 
    dist_pc2_pc1 = pcd2.compute_point_cloud_distance(pcd1)
    dist_pc2_pc1 = np.asarray(dist_pc2_pc1)

    chamferDistance = np.sum((dist_pc2_pc1, dist_pc1_pc2))
    return chamferDistance


In [11]:
mesh1 = o3d.io.read_triangle_mesh("../data/sofa/Sofa.obj")
pcd1 = mesh1.sample_points_uniformly(number_of_points=2500)

mesh2 = o3d.io.read_triangle_mesh("../data/sofa/Sofa_completed_0.obj")
pcd2 = mesh2.sample_points_uniformly(number_of_points=2500)

mesh3 = o3d.io.read_triangle_mesh("../data/sofa/Sofa_completed_1.obj")
pcd3 = mesh3.sample_points_uniformly(number_of_points=2500)

get_chamfer_distance(pcd2,pcd3)


195.9641965506257

In [20]:
import os
from context import generationtools as gnt
import trimesh
import open3d as o3d
import numpy as np

resultPath = "../data/results/"
objectCategory = "03001627"
objectNames = [
    "1a8bbf2994788e2743e99e0cae970928",
    "1a6f615e8b1b5ae4dbbc9440457e303e",
    "1b5e876f3559c231532a8e162f399205",
    "1b05971a4373c7d2463600025db2266",
    "1b80175cc081f3e44e4975e87c20ce53",
    "1bb81d54471d7c1df51f77a6d7299806",
    "1bcd9c3fe6c9087e593ebeeedbff73b"
]

valuesDict = {"id" : "Score"}
pointCount = 10000

def get_chamfer_distance(pcd1,pcd2):
    # Calculate distances of pcd1 to pcd2. 
    dist_pc1_pc2 = pcd1.compute_point_cloud_distance(pcd2)
    dist_pc1_pc2 = np.asarray(dist_pc1_pc2)

    # Calculate distances of pcd2 to pcd1. 
    dist_pc2_pc1 = pcd2.compute_point_cloud_distance(pcd1)
    dist_pc2_pc1 = np.asarray(dist_pc2_pc1)

    chamferDistance = np.mean(dist_pc2_pc1) + np.mean(dist_pc1_pc2)
    return chamferDistance

for objectName in objectNames: 
    print(objectName)
    objectPath = "L:/Recordings/2015-01 ShapeNetCore" + os.sep + objectCategory + os.sep + objectName + "/models/model_normalized_sdf.obj"
    resultObjectPaths = resultPath + objectName + os.sep
    gtMesh = o3d.io.read_triangle_mesh(objectPath)
    gtPcd = gtMesh.sample_points_uniformly(number_of_points=pointCount)

    for root, dirs, files in os.walk(resultObjectPaths):
        for filename in files:
            compObjectPath = os.path.join(root, filename)
            compMesh = o3d.io.read_triangle_mesh(compObjectPath)
            compPcd = compMesh.sample_points_uniformly(number_of_points=pointCount)
            
            chamferDist = get_chamfer_distance(gtPcd,compPcd)

            print("ChamferDist: " + str(chamferDist))

            valuesDict[compObjectPath] = chamferDist

1a8bbf2994788e2743e99e0cae970928
ChamferDist: 0.07971579112819294
ChamferDist: 0.0831483685625774
ChamferDist: 0.08755828267151632
ChamferDist: 0.07246062457024789
ChamferDist: 0.08032405324657216
ChamferDist: 0.08817937653305362
ChamferDist: 0.09493139142567622
ChamferDist: 0.09207609227928165
ChamferDist: 0.0917009288290192
ChamferDist: 0.06232634150428257
ChamferDist: 0.06048148800429729
ChamferDist: 0.06190286051963945
ChamferDist: 0.06341096671193845
ChamferDist: 0.06296408989996669
ChamferDist: 0.06320319857700918
ChamferDist: 0.06248335842540459
ChamferDist: 0.06382499764342466
ChamferDist: 0.06141089848544645
ChamferDist: 0.06191090057146084
ChamferDist: 0.06205670149992305
ChamferDist: 0.06276489297329145
ChamferDist: 0.061844296969618656
ChamferDist: 0.0613873427617604
ChamferDist: 0.06165777657169069
ChamferDist: 0.06129055697441528
ChamferDist: 0.06186025701262709
ChamferDist: 0.06131326772662887
1a6f615e8b1b5ae4dbbc9440457e303e
ChamferDist: 0.12708883060432904
ChamferDist:

In [21]:
import csv

def dict_to_csv(dictionary, filename):
    with open(filename, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(['Key', 'Value'])  # Writing header row
        for key, value in dictionary.items():
            writer.writerow([key, value])

dict_to_csv(valuesDict, 'chamferDist.csv')

### IOU

Intersection over union. Using a volumetric comparison of 2 sdfs.

In [38]:
from context import generationtools as gnt
import trimesh

mesh1 = trimesh.load(r"../data\Shapenet\Shapenet_sdf.obj")
mesh2 = trimesh.load(r"../data\Shapenet\Shapenet_0.obj")

# Normalize the meshes


sdf1 = gnt.mesh_to_sdf_tensor(gnt.scale_mesh_to_unity_cube(mesh1), recenter=False)
sdf2 = gnt.mesh_to_sdf_tensor(gnt.scale_mesh_to_unity_cube(mesh2), recenter=False)

In [2]:
print(gnt.scale_mesh_to_unity_cube(mesh2).bounds)

[[0. 0. 0.]
 [1. 1. 1.]]


In [ ]:
import numpy as np
print(np.sum(sdf1[0] < 0))
print(np.sum(sdf2[0] < 0))

# union = get the closest value per voxel
unionSdf = np.minimum(sdf1[0],sdf2[0])
unionCount = np.sum(unionSdf < 0)
 
# Intersection = get the longest value per voxel
intersectionSdf = np.maximum(sdf1[0],sdf2[0])
intersectionCount = np.sum(intersectionSdf < 0)

iou = intersectionCount/unionCount

print("IOU: " + str(iou))

combined = trimesh.util.concatenate( [ sdf1[1], sdf2[1] ] )
#trimesh.repair.fix_inversion(combined)
combined.show()


In [3]:
import os
from context import generationtools as gnt
import trimesh
import numpy as np

resultPath = "../data/results/"
objectCategory = "03001627"
objectNames = [
    "1a8bbf2994788e2743e99e0cae970928",
    "1a6f615e8b1b5ae4dbbc9440457e303e",
    "1b5e876f3559c231532a8e162f399205",
    "1b05971a4373c7d2463600025db2266",
    "1b80175cc081f3e44e4975e87c20ce53",
    "1bb81d54471d7c1df51f77a6d7299806",
    "1bcd9c3fe6c9087e593ebeeedbff73b"
]

valuesDict = {"id" : "Score"}

for objectName in objectNames: 
    print(objectName)
    objectPath = "L:/Recordings/2015-01 ShapeNetCore" + os.sep + objectCategory + os.sep + objectName + "/models/model_normalized_sdf.obj"
    resultObjectPaths = resultPath + objectName + os.sep
    gtMesh = trimesh.load(objectPath)

    for root, dirs, files in os.walk(resultObjectPaths):
        for filename in files:
            compObjectPath = os.path.join(root, filename)
            compMesh = trimesh.load(compObjectPath)
            sdf1 = gnt.mesh_to_sdf_tensor(gnt.scale_mesh_to_unity_cube(gtMesh), recenter=False)
            sdf2 = gnt.mesh_to_sdf_tensor(gnt.scale_mesh_to_unity_cube(compMesh), recenter=False)

            # union = get the closest value per voxel
            unionSdf = np.minimum(sdf1[0],sdf2[0])
            unionCount = np.sum(unionSdf < 0)
            
            # Intersection = get the longest value per voxel
            intersectionSdf = np.maximum(sdf1[0],sdf2[0])
            intersectionCount = np.sum(intersectionSdf < 0)

            iou = intersectionCount/unionCount

            print("IOU: " + str(iou))

            valuesDict[compObjectPath] = iou


1a8bbf2994788e2743e99e0cae970928
IOU: 0.1755306463152874
IOU: 0.31599057504935363
IOU: 0.21897618797860066
IOU: 0.37619726920725494
IOU: 0.5133094516395701
IOU: 0.29455701819257507
IOU: 0.37256515775034293
IOU: 0.09592879512114719
IOU: 0.5000514509158263
IOU: 0.7052119527449617
IOU: 0.722160644350846
IOU: 0.6399232518476407
IOU: 0.6973127813395171
IOU: 0.6992609826194061
IOU: 0.6933351413655163
IOU: 0.6876205443245946
IOU: 0.6938055499016216
IOU: 0.7304923278307032
IOU: 0.7297138232914
IOU: 0.7291023945938006
IOU: 0.7223040659988215
IOU: 0.6386234463806133
IOU: 0.733440915425805
IOU: 0.7349112426035503
IOU: 0.7281389723667815
IOU: 0.7309325135412091
IOU: 0.7275530430952206
1a6f615e8b1b5ae4dbbc9440457e303e
IOU: 0.05732484076433121
IOU: 0.09837622258095148
IOU: 0.15265519820493642
IOU: 0.10978652619905739
IOU: 0.0953556731334509
IOU: 0.12309583987977321
IOU: 0.1370840594141565
IOU: 0.07760070473292224
IOU: 0.043472906403940886
IOU: 0.23773047118553486
IOU: 0.24111632584646328
IOU: 0.2449

In [7]:
print(valuesDict)
import csv

with open('mycsvfile.csv', 'w') as f:  # You will need 'wb' mode in Python 2.x
    w = csv.DictWriter(f, valuesDict.keys())
    w.writeheader()
    w.writerow(valuesDict)

{'id': 'Score', '../data/results/1a8bbf2994788e2743e99e0cae970928\\25\\0.obj': 0.1755306463152874, '../data/results/1a8bbf2994788e2743e99e0cae970928\\25\\1.obj': 0.31599057504935363, '../data/results/1a8bbf2994788e2743e99e0cae970928\\25\\2.obj': 0.21897618797860066, '../data/results/1a8bbf2994788e2743e99e0cae970928\\25\\3.obj': 0.37619726920725494, '../data/results/1a8bbf2994788e2743e99e0cae970928\\25\\4.obj': 0.5133094516395701, '../data/results/1a8bbf2994788e2743e99e0cae970928\\25\\5.obj': 0.29455701819257507, '../data/results/1a8bbf2994788e2743e99e0cae970928\\25\\6.obj': 0.37256515775034293, '../data/results/1a8bbf2994788e2743e99e0cae970928\\25\\7.obj': 0.09592879512114719, '../data/results/1a8bbf2994788e2743e99e0cae970928\\25\\8.obj': 0.5000514509158263, '../data/results/1a8bbf2994788e2743e99e0cae970928\\50\\0.obj': 0.7052119527449617, '../data/results/1a8bbf2994788e2743e99e0cae970928\\50\\1.obj': 0.722160644350846, '../data/results/1a8bbf2994788e2743e99e0cae970928\\50\\2.obj': 0.6

In [8]:
import csv

def dict_to_csv(dictionary, filename):
    with open(filename, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(['Key', 'Value'])  # Writing header row
        for key, value in dictionary.items():
            writer.writerow([key, value])

dict_to_csv(valuesDict, 'output.csv')

## Material

### Percentage correctness

Since The material prediction is pointwise, we perform a binary true or false percentage prediction check.

In [42]:
import trimesh
import numpy as np

# todo filter by only the filled in points
mesh1 = trimesh.load("../data/sofa/1c17cc67b8c747c3febad4f49b26ec52_normalized-partial-0_color_reconstruction.obj")
mesh2 = trimesh.load("../data/sofa/1c17cc67b8c747c3febad4f49b26ec52_normalized-partial-1_color_reconstruction.obj")
#print(mesh1.vertices.size)
#print(mesh1.colors>125)
#print(mesh2.colors>125)
count = np.sum(np.all(np.equal(mesh1.colors>125, mesh2.colors>125), axis=1))

print("The percentage of correctly segmented points is: " + str(count / (mesh1.vertices.size/3)))

The percentage of correctly segmented points is: 0.96504


## Texture

### Texture similarity

A feature vector is calculated for each patch in the predicted inpainted texture and compared to the ground truth (if it exists). 

In [12]:
import torch

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
efficientnet = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_efficientnet_b0', pretrained=True)
utils = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_convnets_processing_utils')
efficientnet.eval().to(device)
print("Model loaded")

Using cache found in C:\Users\jelle/.cache\torch\hub\NVIDIA_DeepLearningExamples_torchhub
C:\Users\jelle/.cache\torch\hub\NVIDIA_DeepLearningExamples_torchhub\PyTorch\Classification\ConvNets\image_classification\models\common.py:13: UserWarning: pytorch_quantization module not found, quantization will not be available
  warnings.warn(
C:\Users\jelle/.cache\torch\hub\NVIDIA_DeepLearningExamples_torchhub\PyTorch\Classification\ConvNets\image_classification\models\efficientnet.py:17: UserWarning: pytorch_quantization module not found, quantization will not be available
  warnings.warn(


Model loaded


Using cache found in C:\Users\jelle/.cache\torch\hub\NVIDIA_DeepLearningExamples_torchhub


In [13]:
import torchvision.transforms as transforms
from PIL import Image

def load_jpeg_from_file(image, image_size, cuda=True):
    img_transforms = transforms.Compose(
        [
            transforms.Resize(image_size + 32),
            transforms.CenterCrop(image_size),
            transforms.ToTensor(),
        ]
    )

    img = img_transforms(image)
    with torch.no_grad():
        # mean and std are not multiplied by 255 as they are in training script
        # torch dataloader reads data into bytes whereas loading directly
        # through PIL creates a tensor with floats in [0,1] range
        mean = torch.tensor([0.485, 0.456, 0.406]).view(1, 3, 1, 1)
        std = torch.tensor([0.229, 0.224, 0.225]).view(1, 3, 1, 1)

        if cuda:
            mean = mean.cuda()
            std = std.cuda()
            img = img.cuda()
        img = img.float()

        input = img.unsqueeze(0).sub_(mean).div_(std)

    return input

In [22]:
import cv2

image1 = cv2.imread(r'../data\1c17cc67b8c747c3febad4f49b26ec52\fabric_full_gt_texture.png')[:,:3]
image1_rotated = cv2.rotate(image1, cv2.ROTATE_180)
image2 = cv2.imread(r'../data\1c17cc67b8c747c3febad4f49b26ec52\fixedImage_50.png')[:,:3]

imageTensor1 = load_jpeg_from_file(Image.fromarray(image1), image1.shape[0], False)
imageTensor1_rotated = load_jpeg_from_file(Image.fromarray(image1_rotated), image1_rotated.shape[0], False)
imageTensor2 = load_jpeg_from_file(Image.fromarray(image2), image2.shape[0], False)
imageFeatures1 = efficientnet(imageTensor1)
imageFeatures1_rotated = efficientnet(imageTensor1_rotated)
imageFeatures2 = efficientnet(imageTensor2)

print(imageFeatures1.shape)

torch.Size([1, 1000])


In [23]:
cos = torch.nn.CosineSimilarity(dim=1, eps=1e-6)
output = cos(imageFeatures1, imageFeatures2)
output_rotated = cos(imageFeatures1, imageFeatures1_rotated)
print("The Patch similarity of a different is: ")
print(output[0].item())
print("The Patch similarity of a similar image is: ")
print(output_rotated[0].item())

The Patch similarity of a different is: 
0.9004584550857544
The Patch similarity of a similar image is: 
0.9999054670333862
